Project: Tweets analysis of the main Italian populist politicians - Giuseppe Grillo, Giorgia Meloni and Matteo Salvini.

Student: **Alessandro Monolo** | 1790210

Lecturer: Erik HEKMAN

Fundamentals of Data Science - Master Data-Driven Design, Hogeschool Utrecht.

August 2021 - Block E

In [ ]:
# Here I import the libraries I need

In [1]:
import tweepy
import requests
import json
import webbrowser
import time
import pandas as pd

In [ ]:
# Here I save my personal twitter API secret keys

In [2]:
consumer_key = "zEOrE8aIMeJDVYQr09miZYS84"
consumer_secret = "BsRCsGxPue6VgN2PiPcqHAbvOLtdYRynwWYOMXjcodju9Yie5i"
access_token = "1400895858938462211-PQaaX3vGL71KOhDr0lCLNI72c7lMkJ"
access_token_secret = "5xVWuUIAEn4U4DfeeMNeSWPhHPLLNWG2F59tjov4loUKK"

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [4]:
api = tweepy.API(auth, wait_on_rate_limit = True)

In [ ]:
# Declare the twitter user profile where to get the last tweets using tweepy as scraping library

In [5]:
userID = "matteosalvinimi"

In [6]:
tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )

In [ ]:
# Test if everything is working smoothly

In [7]:
for info in tweets[:3]:
     print("ID: {}".format(info.id))
     print(info.created_at)
     print(info.full_text)
     print("\n")

ID: 1423972236021616640
2021-08-07 11:40:11
- #Tortu: “Fino alla fine! 🤣 Grazie Giorgio!”
- #Chiellini: “😅 di nulla!”.
AdORO! 💙🇮🇹🥇
#4x100 https://t.co/adqbgwhIy4


ID: 1423957506146111494
2021-08-07 10:41:39
🔴LIVE DA FIRENZE
Il collegio di Siena dove si candida Letta sarà contendibile. Sulla pelle dei cittadini toscani, e penso al Monte dei Paschi, il Pd ne ha combinate di tutti i colori, distruggendo una banca che ha sei secoli di storia. Conto che i senesi scelgano il futuro. https://t.co/5PkCnzdUHh


ID: 1423957002540228621
2021-08-07 10:39:39
Anche in questo caldo sabato di agosto in tantissimi qui a Firenze, mercato di Campo di Marte, per i #referendumgiustizia. Grazie ai volontari della Lega impegnati ai gazebo anche questo fine settimana in tutta Italia: verso il mezzo milione di firme!
👉https://t.co/JQr12goqKB https://t.co/J9CnKaQpo0




In [8]:
all_tweets = []
all_tweets.extend(tweets)
oldest_id = tweets[-1].id

In [ ]:
# Get the last tweets of userID with tweet_mode set as extended in order to get the full
# text of each tweets and therefor not a cut one which has only 140 characters available per each tweet:

In [9]:
while True:
    tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           max_id = oldest_id - 1,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print('N of tweets downloaded till now {}'.format(len(all_tweets)))

N of tweets downloaded till now 392
N of tweets downloaded till now 591
N of tweets downloaded till now 791
N of tweets downloaded till now 988
N of tweets downloaded till now 1187
N of tweets downloaded till now 1387
N of tweets downloaded till now 1586
N of tweets downloaded till now 1786
N of tweets downloaded till now 1986
N of tweets downloaded till now 2185
N of tweets downloaded till now 2385
N of tweets downloaded till now 2585
N of tweets downloaded till now 2782
N of tweets downloaded till now 2981
N of tweets downloaded till now 3181
N of tweets downloaded till now 3206


In [ ]:
# Store into a csv file the info of each tweets I need, such as text, datetime, number of retweets and favorite count:

In [10]:
from pandas import DataFrame
outtweets = [[tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count, 
              tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx,tweet in enumerate(all_tweets)]
df = DataFrame(outtweets,columns=["id","created_at","favorite_count","retweet_count", "text"])
df.to_csv('%s_tweets.csv' % userID,index=False)
df.head(211)

,id,created_at,favorite_count,retweet_count,text
0,1423972236021616640,2021-08-07 11:40:11,314,31,- #Tortu: “Fino alla fine! 🤣 Grazie Giorgio!”\...
1,1423957506146111494,2021-08-07 10:41:39,201,37,🔴LIVE DA FIRENZE\nIl collegio di Siena dove si...
2,1423957002540228621,2021-08-07 10:39:39,102,22,Anche in questo caldo sabato di agosto in tant...
3,1423931985370783744,2021-08-07 09:00:15,319,27,Non esiste solo il Covid purtroppo. Grazie a R...
4,1423699275071041539,2021-08-06 17:35:32,589,68,"Menagramo al contrario, #Travaglio il 29 lugli..."
...,...,...,...,...,...
206,1414607660192784389,2021-07-12 15:28:42,335,43,Qui Gizzeria (Catanzaro): la Calabria è una me...
207,1414606630864441346,2021-07-12 15:24:37,192,22,"Al di là dei gazebo, vi ricordo che potete and..."
208,1414606620475154437,2021-07-12 15:24:35,297,35,Sorrisi e bellissima accoglienza qui in terra ...
209,1414597917952802821,2021-07-12 14:50:00,755,98,Allora non proprio è sempre #colpadiSalvini\n\...
